In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import bz2
import pybaseball

NOTE: you'll have to download the data from <https://historicdata.betfair.com/> yourself. This will take quite some time because baseball is in "all other sports", which means you have to download very many irrelevant files. Anyway, put those in the relevant directories which the code demands. 

# 1. Get summary of what files are where in betfair data

In [2]:
def bz2_to_json(filepath):
    content_string = bz2.open(filepath, 'rt', encoding='utf-8').read()
    # Instead of a bunch of dicts printed one after the other
    # Reshape so it's a list of dicts
    fixed_string = "[" + content_string.replace("}\n{", "},{") + "]"
    return json.loads(fixed_string)

In [3]:
def get_market_data(filepath):
    try:
        contents = bz2_to_json(filepath)
        market_type = contents[0]["mc"][0]["marketDefinition"]["marketType"]
        event_name = contents[0]["mc"][0]["marketDefinition"]["eventName"]
        market_time = contents[0]["mc"][0]["marketDefinition"]["marketTime"]
        return (filepath, market_type, event_name, market_time)
    except Exception:
        return (filepath, None, None, None)

In [5]:
def get_json_info_from_dir(directory):
    rows = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if not file.endswith(".bz2"):
                continue
            filepath = f"{root}/{file}"
            market_data = get_market_data(filepath)
            rows.append(market_data)
            if len(rows) % 1000 == 0: # print so you can check progress as it goes
                print(len(rows))
    file_df = pd.DataFrame(rows, columns=["filepath", "mkt_type", "name", "time"])
    return file_df

In [6]:
# Go through ENTIRE 3gb of data
# this takes over an hour to run, pls be patient
file_df = get_json_info_from_dir("market_data")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [7]:
# Save this to disk asap, don't want to run that previous cell again
file_df.to_csv("refined_csvs/1_full_file_df.csv", index=False)

# 2. Locate just the MLB files

In [8]:
full_file_df = pd.read_csv("refined_csvs/1_full_file_df.csv")

In [9]:
# Get just the baseball ones
shortened = full_file_df.dropna()[full_file_df.dropna()["name"].str.contains("@")].copy()
shortened["home"] = shortened["name"].str.split(" @ ").str[1]
shortened["away"] = shortened["name"].str.split(" @ ").str[0]
teams = [
    "Los Angeles Dodgers", "New York Yankees", "Cleveland Guardians", "New York Mets", 
    "Minnesota Twins", "Pittsburgh Pirates", "Houston Astros", "Chicago Cubs", 
    "St. Louis Cardinals", "Kansas City Royals", "Atlanta Braves", "Philadelphia Phillies", 
    "Detroit Tigers", "Colorado Rockies", "Arizona Diamondbacks", "San Diego Padres", 
    "Baltimore Orioles", "Oakland Athletics", "Texas Rangers", "Milwaukee Brewers", 
    "Washington Nationals", "Miami Marlins", "Tampa Bay Rays", "Boston Red Sox", 
    "Toronto Blue Jays", "Chicago White Sox", "Los Angeles Angels", "San Francisco Giants", 
    "Seattle Mariners", "Cincinnati Reds"
]
mlb_df = shortened[
    shortened["home"].isin(teams) 
    & shortened["away"].isin(teams) 
    & (shortened["mkt_type"] == "MATCH_ODDS")
].copy()

There's a lot of double counting going on here!! Look at this:

In [10]:
mlb_df[(mlb_df["home"] == "Miami Marlins") | (mlb_df["away"] == "Miami Marlins")]

,filepath,mkt_type,name,time,home,away
481,market_data/2022/BASIC/2022/Aug/20/31673119/1....,MATCH_ODDS,Miami Marlins @ Los Angeles Dodgers,2022-08-20T02:11:00.000Z,Los Angeles Dodgers,Miami Marlins
1054,market_data/2022/BASIC/2022/Aug/27/31694388/1....,MATCH_ODDS,Los Angeles Dodgers @ Miami Marlins,2022-08-26T22:41:00.000Z,Miami Marlins,Los Angeles Dodgers
1578,market_data/2022/BASIC/2022/Aug/11/31653416/1....,MATCH_ODDS,Miami Marlins @ Philadelphia Phillies,2022-08-11T17:06:00.000Z,Philadelphia Phillies,Miami Marlins
1620,market_data/2022/BASIC/2022/Aug/11/31651076/1....,MATCH_ODDS,Miami Marlins @ Philadelphia Phillies,2022-08-10T23:06:00.000Z,Philadelphia Phillies,Miami Marlins
1807,market_data/2022/BASIC/2022/Aug/7/31644293/1.2...,MATCH_ODDS,Miami Marlins @ Chicago Cubs,2022-08-07T18:21:00.000Z,Chicago Cubs,Miami Marlins
...,...,...,...,...,...,...
574317,market_data/2023/BASIC 6/2023/Apr/5/32240904/3...,MATCH_ODDS,Minnesota Twins @ Miami Marlins,2023-04-04T22:41:00.000Z,Miami Marlins,Minnesota Twins
574318,market_data/2023/BASIC 6/2023/Apr/5/32240904/1...,MATCH_ODDS,Minnesota Twins @ Miami Marlins,2023-04-04T22:41:00.000Z,Miami Marlins,Minnesota Twins
574330,market_data/2023/BASIC 6/2023/Apr/5/32243604/1...,MATCH_ODDS,Minnesota Twins @ Miami Marlins,2023-04-05T17:11:00.000Z,Miami Marlins,Minnesota Twins
576931,market_data/2023/BASIC 6/2023/Apr/25/32296193/...,MATCH_ODDS,Miami Marlins @ Atlanta Braves,2023-04-24T23:21:00.000Z,Atlanta Braves,Miami Marlins


Manual inspection of the files shows the ones that start with "33" are sus and the ones that start with "1.2" are fine, or just "1" generally

In [11]:
mlb_df = mlb_df[mlb_df["filepath"].str.split("/").str[-1].str[0] == "1"].copy()
mlb_df["filesize"] = mlb_df["filepath"].apply(os.path.getsize)
mlb_df = mlb_df.sort_values( # More dropping duplicates, keeping larger file now
    by="filesize", ascending=False
).drop_duplicates(subset=["name", "time"], keep="first")
mlb_df.to_csv("refined_csvs/2_mlb_file_df.csv", index=False)

# 3. Add in market data

In [12]:
mlb_df = pd.read_csv("refined_csvs/2_mlb_file_df.csv")

In [13]:
def process_game(filename, lookback):
    data = bz2_to_json(filename)
    
    # Get the time
    game_time = data[0]["mc"][0]["marketDefinition"]["suspendTime"]
    runners = data[0]["mc"][0]["marketDefinition"]["runners"]
    away_name, away_id = runners[0]["name"], runners[0]["id"]
    home_name, home_id = runners[1]["name"], runners[1]["id"]
    
    # Get the trades
    # home_trade_rows = []
    # away_trade_rows = []
    home_trades = []
    away_trades = []
    for line in data[2:]:
        if "rc" in line["mc"][0]:
            curr_time = line["pt"]
            for trade in line["mc"][0]["rc"]:
                if trade["id"] == home_id:
                    # home_trade_rows.append([curr_time, trade["ltp"]])
                    home_trades.append(trade["ltp"])
                elif trade["id"] == away_id:
                    # away_trade_rows.append([curr_time, trade["ltp"]])
                    away_trades.append(trade["ltp"])
                else:
                    print("Questionable")
        try:
            if line["mc"][0]["marketDefinition"]["turnInPlayEnabled"]:
                break
        except KeyError:
            pass
    # home_trade_df = pd.DataFrame(home_trade_rows, columns=["time", "price"])
    # away_trade_df = pd.DataFrame(away_trade_rows, columns=["time", "price"])
    
    # Now get the winner
    end_runners = data[-1]["mc"][0]["marketDefinition"]["runners"]
    if end_runners[0]["status"] == "WINNER":
        winner = away_name
    elif end_runners[1]["status"] == "WINNER":
        winner = home_name
    else:
        winner = None
    
    return {
        "home_ask": 1 / min(home_trades[-lookback:]) if len(home_trades) >= 5 else None,
        "home_bid": 1 / max(home_trades[-lookback:]) if len(home_trades) >= 5 else None,
        "away_ask": 1 / min(away_trades[-lookback:]) if len(away_trades) >= 5 else None,
        "away_bid": 1 / max(away_trades[-lookback:]) if len(away_trades) >= 5 else None,
        "home_trades": home_trades,
        "away_trades": away_trades,
        "winner": winner, 
        "game_time": game_time
    }

In [14]:
new_df = pd.DataFrame(
    mlb_df["filepath"].apply(lambda x: process_game(x, 5)).to_list(), 
    index=mlb_df.index
)

There is a lot of missing data, which is NOT good. Oh no! Still, having a look manually suggests those are the low-interest ones anywhere where there isn't much of a market. 

In [15]:
new_df.isna().mean()

home_ask       0.104774
home_bid       0.104774
away_ask       0.102710
away_bid       0.102710
home_trades    0.000000
away_trades    0.000000
winner         0.022452
game_time      0.000000
dtype: float64

In [16]:
df = pd.concat([mlb_df, new_df], axis=1)

# Add in a bunch of useful columns
week_map = dict(zip(range(7), ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]))
df["game_time"] = pd.to_datetime(df["game_time"]).dt.tz_convert("-05:00")
df["weekday"] = df["game_time"].dt.dayofweek.map(week_map)

df["synth_home_bid"] = pd.DataFrame({0: df["home_bid"], 1: 1 - df["away_ask"]}).max(axis=1)
df["synth_home_ask"] = pd.DataFrame({0: df["home_ask"], 1: 1 - df["away_bid"]}).max(axis=1)
df["synth_home_mid"] = (df["synth_home_bid"] + df["synth_home_ask"]) / 2
df["home_win"] = df["home"] == df["winner"]

df["anti_home_pnl"] = df["synth_home_bid"] - df["home_win"] # bid
df["pro_home_pnl"] = df["home_win"] - df["synth_home_ask"] # ask

df["date_mod_year"] = pd.to_datetime("2000" + df["time"].str[4:])

df.to_csv("refined_csvs/3_market_data_df.csv", index=False)

# 4. Import pybaseball data

In [17]:
df = pd.read_csv("refined_csvs/3_market_data_df.csv")

In [18]:
large_mkt = pd.read_csv("csvs/large_market.csv")
large_mkt.loc[29, "team"] = "wsn" # ehh little correction
large_mkt.loc[1, "team"] = "oak"
code_to_team = dict(zip(large_mkt["team"].str.upper(), large_mkt["full_name"]))
team_to_code = dict(zip(large_mkt["full_name"], large_mkt["team"].str.upper()))

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
month_to_num = dict(zip(months, [f"{i:02d}" for i in range(1, 13)]))

In [19]:
def transform_team_df(team_df, year):
    # Takes the info from pybaseball df and repackages it more usefully for my purposes
    trans_df = pd.DataFrame()
    trans_df["name"] = pd.Series(np.where(
        team_df["Home_Away"] == "Home",
        team_df["Opp"].map(code_to_team) + " @ " + team_df["Tm"].map(code_to_team),
        team_df["Tm"].map(code_to_team) + " @ " + team_df["Opp"].map(code_to_team)
    ), index=team_df.index)
    trans_df["date"] = (
        str(year) 
        + "-" + team_df["Date"].str.split(" ").str[1].map(month_to_num)
        + "-" + team_df["Date"].str.split(" ").str[2].str.zfill(2)
    )
    trans_df["home_win"] = np.where(
        team_df["Home_Away"] == "Home",
        team_df["W/L"].str[0] ==  "W",
        team_df["W/L"].str[0] == "L"
    )
    trans_df["weekday"] = team_df["Date"].str[:3]
    trans_df["night"] = team_df["D/N"] == "N"
    trans_df["cli"] = team_df["cLI"] # championship leverage index; measures importance
    trans_df["home_pitcher"] = np.where(
        (team_df["Home_Away"] == "Home") == (team_df["W/L"].str[0] == "W"),
        team_df["Win"],
        team_df["Loss"]
    )
    trans_df["away_pitcher"] = np.where(
        (team_df["Home_Away"] == "Home") == (team_df["W/L"].str[0] == "W"),
        team_df["Loss"],
        team_df["Win"]
    )
    trans_df["attendance"] = team_df["Attendance"]
    trans_df["team"] = team_df["Tm"].map(code_to_team)
    
    return trans_df.drop(["home_pitcher", "away_pitcher"], axis=1)

This following two cells takes a little while to run - it does 30 queries per year and the time expense of figuring out to run pybaseball queries asynchronously is much more than the time I will save, so just be content to wait. Ignore all the warnings too, that's from the pybaseball library so it's not my fault. (Great library though!)

In [20]:
team_df_dict = {}

In [21]:
for team_code in code_to_team.keys():
    for year in [2022, 2023, 2024]:
        tag = f"{team_code.lower()}_{year}"
        if tag in team_df_dict:
            # This accounts for trash library wifi cutting off
            # so you can now do it in bits
            print(f"Skipping {tag}")
            continue
        team_df = pybaseball.schedule_and_record(year, team_code)
        team_df_dict[tag] = team_df

http://www.baseball-reference.com/teams/ARI/2022-schedule-scores.shtml


/Users/markrothery/miniforge3/envs/fastai_env/lib/python3.12/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


http://www.baseball-reference.com/teams/ARI/2023-schedule-scores.shtml


/Users/markrothery/miniforge3/envs/fastai_env/lib/python3.12/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


http://www.baseball-reference.com/teams/ARI/2024-schedule-scores.shtml


...

/Users/markrothery/miniforge3/envs/fastai_env/lib/python3.12/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


http://www.baseball-reference.com/teams/WSN/2023-schedule-scores.shtml


/Users/markrothery/miniforge3/envs/fastai_env/lib/python3.12/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


http://www.baseball-reference.com/teams/WSN/2024-schedule-scores.shtml


/Users/markrothery/miniforge3/envs/fastai_env/lib/python3.12/site-packages/pybaseball/team_results.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Attendance'].replace(r'^Unknown$', np.nan, regex=True, inplace = True) # make this a NaN so the column can benumeric


In [22]:
transformed = []
for team_code in code_to_team.keys():
    for year in [2022, 2023, 2024]:
        tag = f"{team_code.lower()}_{year}"
        team_df_dict[tag]["year"] = year
        transformed.append(transform_team_df(team_df_dict[tag], year))

In [23]:
big_pyball_df = pd.concat(transformed)
big_pyball_df["home"] = big_pyball_df["name"].str.split(" @ ").str[1]
big_pyball_df["away"] = big_pyball_df["name"].str.split(" @ ").str[0]
big_pyball_df["home_cli"] = np.where(
    big_pyball_df["team"] == big_pyball_df["home"], 
    big_pyball_df["cli"], 
    np.nan
)
big_pyball_df["away_cli"] = np.where(
    big_pyball_df["team"] == big_pyball_df["away"], 
    big_pyball_df["cli"], 
    np.nan
)
big_pyball_df.sort_values(
    by=["date", "name", "night", "attendance", "home_cli"], 
    inplace=True
)

In [24]:
# This is a bit of nightmarish manipulation to get home cLI and away cLI correct
home_cli_df = big_pyball_df.drop_duplicates(
    subset=["date", "name", "night", "attendance"], keep="first"
)
away_cli_df = big_pyball_df.drop_duplicates(
    subset=["date", "name", "night", "attendance"], keep="last"
)
trim_pyball_df = big_pyball_df.drop_duplicates(subset=["date", "name", "night", "attendance"]).copy()
trim_pyball_df["home_cli"] = home_cli_df["home_cli"].to_list()
trim_pyball_df["away_cli"] = away_cli_df["away_cli"].to_list()
trim_pyball_df.drop("team", axis=1, inplace=True)

In [26]:
trim_pyball_df.to_csv("refined_csvs/4_pyball_df.csv", index=False)

# 5. Combining the information

In [27]:
market_df = pd.read_csv("refined_csvs/3_market_data_df.csv")
pyball_df = pd.read_csv("refined_csvs/4_pyball_df.csv")

In [33]:
market_df["date"] = market_df["game_time"].str[:10]

In [34]:
combined = market_df.merge(
    pyball_df.drop("home_win", axis=1), 
    on=["name", "date", "home", "away", "weekday"]
).drop_duplicates(subset=["date", "name"], keep=False)
# NOTE: drop home_win because if you merge on it, you only get doubleheaders when they have the opposite result
# and if you don't merge on it, you get two columns which is yuck
# so just keeping the first game of doubleheaders

In [51]:
# Here backslashing because "." on its own is regex :(
# Ignore the warnings, it works fine
combined["num_home_trades"] = combined["home_trades"].str.count("\.")
combined["num_away_trades"] = combined["away_trades"].str.count("\.")

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
/var/folders/fs/kf4nq2wx2nb97jc1q5508r840000gn/T/ipykernel_65894/2285439671.py:3: SyntaxWarning: invalid escape sequence '\.'
  combined["num_home_trades"] = combined["home_trades"].str.count("\.")
/var/folders/fs/kf4nq2wx2nb97jc1q5508r840000gn/T/ipykernel_65894/2285439671.py:4: SyntaxWarning: invalid escape sequence '\.'
  combined["num_away_trades"] = combined["away_trades"].str.count("\.")


In [50]:
combined.to_csv("refined_csvs/5_enhanced.csv", index=False)